In [1]:
import pandas as pd
import hashlib
import numpy as np

In [2]:
data_path = "../../../database"

df = pd.read_csv(f"{data_path}/silver/arquivo_anonimizado_v2.csv")

In [3]:
def processar_disciplina(disciplina):
    detalhes_disciplinas = disciplina.split(" - ")
    if len(detalhes_disciplinas) == 5:
        detalhes_disciplinas[1] = " - ".join(detalhes_disciplinas[1:3])
        del detalhes_disciplinas[2]
    return tuple(detalhes_disciplinas)

In [4]:
def pegar_disciplinas(x):
    if x[0] == 'nan':
        return np.nan
    try:
        return (
            x[1].split()[0].replace(" ", ""),
            " ".join(x[1].split()[1:]),
        )
    except IndexError:
        print("Erro de índice para:", x)
        return None

In [5]:
# Converte a coluna 'disciplinas_cursadas' para string
df["disciplinas_cursadas"] = df["disciplinas_cursadas"].astype(str)

# Processa a coluna 'disciplinas_cursadas' para separar em periodo, disciplina, grau, stituação
df["disciplinas_cursadas"] = df["disciplinas_cursadas"].apply(lambda row: [processar_disciplina(item) for item in row.split("\n")])

# Explode a coluna 'disciplinas_cursadas' para dividir as listas em linhas separadas
df_exploded = df.explode("disciplinas_cursadas")

df_exploded["disciplinas_cursadas"] = df_exploded["disciplinas_cursadas"].apply(pegar_disciplinas)

df_exploded.dropna(inplace=True)

In [6]:
df_disciplina = pd.DataFrame(
    df_exploded["disciplinas_cursadas"].tolist(),
    index=df_exploded.index,
    columns=(
        "cod_disciplina",
        "nome_disciplina",
    ),
)

d_disciplina = df_disciplina.drop_duplicates().reset_index(drop=True)

In [7]:
d_disciplina["sk_d_disciplina"] = d_disciplina.apply(
    lambda x: hashlib.sha256(
        str(x["cod_disciplina"] + x["nome_disciplina"]).encode()
    ).hexdigest(),
    axis=1,
)

colunas_disciplina = ["sk_d_disciplina", "cod_disciplina", "nome_disciplina"]

d_disciplina = d_disciplina.reindex(columns=colunas_disciplina)

In [8]:
d_disciplina.to_csv(f"{data_path}/gold/d_disciplina.csv", index=False)